# Text summary using libraries for NLP: nltk and SpaCy

### Import of necessary packages

In [3]:
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.probability import FreqDist
import string
from heapq import nlargest
from collections import defaultdict

### Preparation of the text

In [16]:
# Text to process
text = 'The Orbiter Discovery, OV-103, is considered eligible for listing in the National Register of Historic Places (NRHP) in the context of the U.S. Space Shuttle Program (1969-2011) under Criterion A in the areas of Space Exploration and Transportation and under Criterion C in the area of Engineering. Because it has achieved significance within the past fifty years, Criteria Consideration G applies. Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program to date; she was the third of five orbiters built by NASA. Unlike the Mercury, Gemini, and Apollo programs, the SSP’s emphasis was on cost effectiveness and reusability, and eventually the construction of a space station. Including her maiden voyage (launched August 30, 1984), Discovery flew to space thirty-nine times, more than any of the other four orbiters; she was also the first orbiter to fly twenty missions. She had the honor of being chosen as the Return to Flight vehicle after both the Challenger and Columbia accidents. Discovery was the first shuttle to fly with the redesigned SRBs, a result of the Challenger accident, and the first shuttle to fly with the Phase II and Block I SSME. Discovery also carried the Hubble Space Telescope to orbit and performed two of the five servicing missions to the observatory. She flew the first and last dedicated Department of Defense (DoD) missions, as well as the first unclassified defense-related mission. In addition, Discovery was vital to the construction of the International Space Station (ISS); she flew thirteen of the thirty-seven total missions flown to the station by a U.S. Space Shuttle. She was the first orbiter to dock to the ISS, and the first to perform an exchange of a resident crew. Under Criterion C, Discovery is significant as a feat of engineering. According to Wayne Hale, a flight director from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological leap from expendable rockets and capsules to a reusable, winged, hypersonic, cargo-carrying spacecraft.” Although her base structure followed a conventional aircraft design, she used advanced materials that both minimized her weight for cargo-carrying purposes and featured low thermal expansion ratios, which provided a stable base for her Thermal Protection System (TPS) materials. The Space Shuttle orbiter also featured the first reusable TPS; all previous spaceflight vehicles had a single-use, ablative heat shield. Other notable engineering achievements of the orbiter included the first reusable orbital propulsion system, and the first two-fault-tolerant Integrated Avionics System. As Hale stated, the Space Shuttle remains “the largest, fastest, winged hypersonic aircraft in history,” having regularly flown at twenty-five times the speed of sound.'

### Text summary using nltk

In [7]:
class NLTKProcessor:
    def __init__(self, text):
        self.text = text
        nltk.download('punkt')
        nltk.download('stopwords')
        self.stop_words = set(stopwords.words('english'))
        self.punctuation = string.punctuation + '\n'

    def prepare_text(self):
        words = word_tokenize(self.text)
        words = [word.lower() for word in words if word.isalnum()]
        words = [word for word in words if word not in self.stop_words]
        return words

    def generate_summary(self, num_sentences=3):
        prepared_text = self.prepare_text()
        freq_dist = FreqDist(prepared_text)
        ranking = {}
        for i, sentence in enumerate(sent_tokenize(self.text)):
            for word in word_tokenize(sentence.lower()):
                if word in freq_dist:
                    if i not in ranking:
                        ranking[i] = freq_dist[word]
                    else:
                        ranking[i] += freq_dist[word]

        top_sentences = nlargest(num_sentences, ranking, key=ranking.get)
        summary = [sent_tokenize(self.text)[i] for i in sorted(top_sentences)]
        return ' '.join(summary)


In [15]:
nltk_processor = NLTKProcessor(text)
prepared_text = nltk_processor.prepare_text()
print(f'Tokens:\n{prepared_text}\n')
nltk_summary = nltk_processor.generate_summary()
print(f'NLTK Summary:\n{nltk_summary}')

Tokens:
['orbiter', 'discovery', 'considered', 'eligible', 'listing', 'national', 'register', 'historic', 'places', 'nrhp', 'context', 'space', 'shuttle', 'program', 'criterion', 'areas', 'space', 'exploration', 'transportation', 'criterion', 'c', 'area', 'engineering', 'achieved', 'significance', 'within', 'past', 'fifty', 'years', 'criteria', 'consideration', 'g', 'applies', 'criterion', 'discovery', 'significant', 'oldest', 'three', 'extant', 'orbiter', 'vehicles', 'constructed', 'space', 'shuttle', 'program', 'ssp', 'longest', 'running', 'american', 'space', 'program', 'date', 'third', 'five', 'orbiters', 'built', 'nasa', 'unlike', 'mercury', 'gemini', 'apollo', 'programs', 'ssp', 'emphasis', 'cost', 'effectiveness', 'reusability', 'eventually', 'construction', 'space', 'station', 'including', 'maiden', 'voyage', 'launched', 'august', '30', '1984', 'discovery', 'flew', 'space', 'times', 'four', 'orbiters', 'also', 'first', 'orbiter', 'fly', 'twenty', 'missions', 'honor', 'chosen', 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Text summary using SpaCy

In [17]:
class SpaCyProcessor:
    def __init__(self, text):
        self.text = text
        self.nlp = spacy.load('en_core_web_sm')
        nltk.download('punkt')
        nltk.download('stopwords')
        self.stop_words = set(stopwords.words('english'))
        self.punctuation = string.punctuation + '\n'

    def prepare_text(self):
        doc = self.nlp(self.text)
        tokens = [token.text for token in doc]
        tokens = [token.lower() for token in tokens if token.isalnum()]
        tokens = [token for token in tokens if token not in self.stop_words]
        return tokens

    def generate_summary(self, prepared_text, num_sentences=3):
        freq_dist = FreqDist(prepared_text)
        ranking = {}
        for i, sentence in enumerate(sent_tokenize(self.text)):
            doc = self.nlp(sentence)
            for token in doc:
                if token.text.lower() in freq_dist:
                    if i not in ranking:
                        ranking[i] = freq_dist[token.text.lower()]
                    else:
                        ranking[i] += freq_dist[token.text.lower()]

        top_sentences = nlargest(num_sentences, ranking, key=ranking.get)
        summary = [sent_tokenize(self.text)[i] for i in sorted(top_sentences)]
        return ' '.join(summary)


In [19]:
spacy_processor = SpaCyProcessor(text)
spacy_summary = spacy_processor.generate_summary(prepared_text)
print(f'SpaCy Summary:\n{spacy_summary}')

SpaCy Summary:
The Orbiter Discovery, OV-103, is considered eligible for listing in the National Register of Historic Places (NRHP) in the context of the U.S. Space Shuttle Program (1969-2011) under Criterion A in the areas of Space Exploration and Transportation and under Criterion C in the area of Engineering. Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program to date; she was the third of five orbiters built by NASA. According to Wayne Hale, a flight director from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological leap from expendable rockets and capsules to a reusable, winged, hypersonic, cargo-carrying spacecraft.” Although her base structure followed a conventional aircraft design, she used advanced materials that both minimized her weight for cargo-carrying purposes and featured low thermal expansion ratios, which p

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
